In [17]:
! pip install tensorboard

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.7/151.7 kB 1.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 1.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 2.1 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 186.4/186.4 kB 2.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 1.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.3/400.3 kB 1.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 1.7 MB/s eta 0:00:0000:0100:01


In [1]:
import os

# Check if the notebook is running on Colab
if 'COLAB_GPU' in os.environ:
    # This block will run only in Google Colab
    IN_COLAB = True
    print("Running on Google Colab. Cloning the repository.")
    !git clone https://github.com/pedro15sousa/R244-wm-load-balancing.git
    %cd R244-wm-load-balancing/notebooks
else: 
    # This block will run if not in Google Colab
    IN_COLAB = False
    print("Not running on Google Colab. Assuming local environment.")

Not running on Google Colab. Assuming local environment.


In [2]:
import sys
sys.path.append('..')  # This adds the parent directory (main_folder) to the Python path

In [18]:
import argparse
from os.path import join, exists
from os import mkdir

import torch
import torch.utils.data
from torch import optim
from torch.nn import functional as F
from torchvision import transforms
from torchvision.utils import save_image
from torch.utils.tensorboard import SummaryWriter

from models.vae import VAE

from utils.misc import save_checkpoint
from utils.misc import LSIZE, RED_SIZE, ASIZE
## WARNING : THIS SHOULD BE REPLACE WITH PYTORCH 0.5
from utils.learning import EarlyStopping
from utils.learning import ReduceLROnPlateau
from data.loaders import RolloutObservationDataset

## Initial Setup

In [12]:
# parser = argparse.ArgumentParser(description='VAE Trainer')
# parser.add_argument('--batch-size', type=int, default=32, metavar='N',
#                     help='input batch size for training (default: 32)')
# parser.add_argument('--epochs', type=int, default=1000, metavar='N',
#                     help='number of epochs to train (default: 1000)')
# parser.add_argument('--logdir', type=str, help='Directory where results are logged')
# parser.add_argument('--noreload', action='store_true',
#                     help='Best model is not reloaded if specified')
# parser.add_argument('--nosamples', action='store_true',
#                     help='Does not save samples during training if specified')

# args = parser.parse_args()

args = {
    'batch_size': 32,   # input batch size for training (default: 32)
    'epochs': 30,     # number of epochs to train (default: 1000)
    'logdir': '../exp_dir',  # Directory where results are logged
    'noreload': False,  # Set True if best model is not to be reloaded
    'nosamples': False,  # Set True if samples are not to be saved during training
}

In [6]:
cuda = torch.cuda.is_available()

torch.manual_seed(123)
# Fix numeric divergence due to bug in Cudnn
torch.backends.cudnn.benchmark = True

device = torch.device("cuda" if cuda else "cpu")

# 51 is the number of dimensions in my space: 50 servers + 1 job size
model = VAE(ASIZE+1, LSIZE).to(device)
optimizer = optim.Adam(model.parameters())
scheduler = ReduceLROnPlateau(optimizer, 'min', factor=0.5, patience=5)
earlystopping = EarlyStopping('min', patience=30)

In [13]:
def load_data():
    dataset_train = RolloutObservationDataset('../datasets/loadbalancing', train=True)
    dataset_test = RolloutObservationDataset('../datasets/loadbalancing',train=False)

    train_loader = torch.utils.data.DataLoader(
        dataset_train, batch_size=args['batch_size'], shuffle=True, num_workers=2)
    test_loader = torch.utils.data.DataLoader(
        dataset_test, batch_size=args['batch_size'], shuffle=True, num_workers=2)

    return dataset_test, dataset_train, test_loader, train_loader

## Loss Function

In [8]:
# Reconstruction + KL divergence losses summed over all elements and batch
def loss_function(recon_x, x, mu, logsigma):
    """ VAE loss function """
    BCE = F.mse_loss(recon_x, x, size_average=False)
    # print(mu)
    # print(logsigma)
    # print(recon_x)
    # print(x)

    # see Appendix B from VAE paper:
    # Kingma and Welling. Auto-Encoding Variational Bayes. ICLR, 2014
    # https://arxiv.org/abs/1312.6114
    # 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
    KLD = -0.5 * torch.sum(1 + 2 * logsigma - mu.pow(2) - (2 * logsigma).exp())
    return BCE + KLD

## Training Method

In [9]:
def train(epoch):
    """ One training epoch """
    model.train()
    dataset_train.load_next_buffer()
    train_loss = 0
    for batch_idx, data in enumerate(train_loader):
        data = data.float().to(device)
        optimizer.zero_grad()
        recon_batch, mu, logvar = model(data)
        loss = loss_function(recon_batch, data, mu, logvar)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        if batch_idx % 20 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader),
                loss.item() / len(data)))

    print('====> Epoch: {} Average loss: {:.4f}'.format(
        epoch, train_loss / len(train_loader.dataset)))

## Testing Method

In [10]:
def test():
    """ One test epoch """
    model.eval()
    dataset_test.load_next_buffer()
    test_loss = 0
    with torch.no_grad():
        for data in test_loader:
            data = data.float().to(device)
            recon_batch, mu, logvar = model(data)
            test_loss += loss_function(recon_batch, data, mu, logvar).item()

    test_loss /= len(test_loader.dataset)
    print('====> Test set loss: {:.4f}'.format(test_loss))
    return test_loss

## Main Loop

In [19]:
if IN_COLAB:
    %reload_ext tensorboard
    %tensorboard --logdir ../exp_dir/MNIST/vae/tensorboard

In [21]:
if __name__ == "__main__":
    # Initialize the TensorBoard writer
    logdir = args['logdir']
    writer = SummaryWriter(os.path.join(logdir, 'vae/tensorboard'))

    # load data
    dataset_test, dataset_train, test_loader, train_loader = load_data()

    # check vae dir exists, if not, create it
    vae_dir = join(args['logdir'], 'vae')
    if not exists(vae_dir):
        mkdir(vae_dir)
        mkdir(join(vae_dir, 'samples'))

    reload_file = join(vae_dir, 'best.tar')
    if not args['noreload'] and exists(reload_file):
        state = torch.load(reload_file)
        print("Reloading model at epoch {}"
            ", with test error {}".format(
                state['epoch'],
                state['precision']))
        model.load_state_dict(state['state_dict'])
        optimizer.load_state_dict(state['optimizer'])
        scheduler.load_state_dict(state['scheduler'])
        earlystopping.load_state_dict(state['earlystopping'])


    cur_best = None

    for epoch in range(1, args['epochs'] + 1):
        train(epoch)
        test_loss = test()

        # TensorBoard logging
        writer.add_scalar('Loss/train', train_loss, epoch)
        writer.add_scalar('Loss/test', test_loss, epoch)

        scheduler.step(test_loss)
        earlystopping.step(test_loss)

        # checkpointing
        best_filename = join(vae_dir, 'best.tar')
        filename = join(vae_dir, 'checkpoint.tar')
        is_best = not cur_best or test_loss < cur_best
        if is_best:
            cur_best = test_loss

        save_checkpoint({
            'epoch': epoch,
            'state_dict': model.state_dict(),
            'precision': test_loss,
            'optimizer': optimizer.state_dict(),
            'scheduler': scheduler.state_dict(),
            'earlystopping': earlystopping.state_dict()
        }, is_best, filename, best_filename)

        if not args['nosamples']:
            with torch.no_grad():
                sample = torch.randn(RED_SIZE, LSIZE).to(device)
                sample = model.decoder(sample).cpu()
                save_image(sample.view(64, 3, RED_SIZE, RED_SIZE),
                        join(vae_dir, 'samples/sample_' + str(epoch) + '.png'))

        if earlystopping.stop:
            print("End of Training because of early stopping at epoch {}".format(epoch))
            break

Loading file buffer ...: 100%|██████████| 200/200 
Loading file buffer ...: 100%|██████████| 200/200 


Reloading model at epoch 7, with test error 7.604264052124023


Loading file buffer ...: 100%|██████████| 200/200 


KeyboardInterrupt: 